# sktime Series Annotation (Anomalies, Change Points, Segmentation)

Series annotation attaches labels to time points or intervals: where behavior shifts, where anomalies appear, or how regimes segment across time.

## What is series annotation?
We learn a label function over time:

$$
a_t = g(y_{1:T})
$$

where $a_t$ can be binary (anomaly vs normal), categorical (regime labels), or interval-based (start/end indices).

## Tasks and notation
- **Point anomalies**: $a_t \in \{0,1\}$ flags outlier timestamps.
- **Collective anomalies**: detect abnormal *intervals* $[s_j, e_j]$.
- **Change points**: detect $\tau_1, \tau_2, \dots$ where the data-generating process shifts.
- **Segmentation**: assign regime labels $\ell_t \in \{1,\dots,K\}$ for contiguous regions.

A simple change point model assumes

$$
y_t \sim P_k \quad \text{for} \quad \tau_{k-1} < t \le \tau_k.
$$


In [ ]:
import numpy as np
import plotly.graph_objects as go

rng = np.random.default_rng(9)
t = np.arange(220)
y = 0.6 * np.sin(t / 10) + 0.25 * rng.normal(size=t.size)

# Regime shifts
y[80:] += 1.2
y[150:] -= 1.6
change_points = [80, 150]

# Point anomalies
anoms = np.array([30, 112, 190])
y[anoms] += np.array([3.5, -3.0, 4.0])

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=y, mode="lines", name="series"))
fig.add_trace(
    go.Scatter(
        x=anoms,
        y=y[anoms],
        mode="markers",
        marker=dict(color="crimson", size=9),
        name="point anomalies",
    )
)
for cp in change_points:
    fig.add_vline(x=cp, line_dash="dash", line_color="gray")

fig.update_layout(
    title="Synthetic series with anomalies and change points",
    height=320,
    margin=dict(l=20, r=20, t=50, b=20),
)
fig

## sktime mapping
sktime exposes series annotation via the **series-annotator** scitype. Annotators typically provide:
- `fit` to learn thresholds or model parameters,
- `predict` (or `transform`) to return anomaly labels or change-point intervals.

Estimator tags describe whether an annotator supports multivariate series, returns point labels vs intervals, or requires exogenous variables.

## Evaluation notes
- Use tolerance windows for change points so near-misses count.
- For anomalies, prefer precision/recall/F1 over raw accuracy when events are rare.
- For segmentation, compare overlap of predicted vs true intervals (IoU-style metrics).

## Enhanced Mathematical Foundation

### CUSUM (Cumulative Sum) Statistic
The CUSUM statistic detects shifts in the mean by accumulating deviations:

$$S_t = \max(0, S_{t-1} + (x_t - \mu_0) - k)$$

where $\mu_0$ is the target mean and $k$ is the allowable drift. A change is signaled when $S_t > h$ (threshold).

### Binary Segmentation Cost Function
Binary segmentation recursively partitions the series by minimizing within-segment variance:

$$C(y_{a:b}) = \sum_{t=a}^{b}(y_t - \bar{y}_{a:b})^2$$

The optimal split point $\tau^*$ minimizes $C(y_{1:\tau}) + C(y_{\tau+1:n})$.

### PELT (Pruned Exact Linear Time) Penalty
PELT finds the optimal segmentation by minimizing:

$$\sum_{i=1}^{m+1} C(y_{\tau_{i-1}:\tau_i}) + \beta m$$

where $m$ is the number of change points and $\beta$ is the penalty per change point (controls model complexity).

### Z-Score Anomaly Detection
For point anomaly detection using rolling statistics:

$$z_t = \frac{y_t - \mu}{\sigma}$$

A point is flagged as anomalous if $|z_t| > \tau$ where $\tau$ is typically 2-3 standard deviations.

## Low-Level NumPy Implementation

Core algorithms for change point detection, segmentation, and anomaly detection built from scratch.

In [ ]:
def segment_cost(y: np.ndarray, start: int, end: int) -> float:
    """
    Compute variance-based cost for a segment y[start:end].
    
    Cost function: C(y_{a:b}) = sum((y_t - mean(y_{a:b}))^2)
    
    Parameters
    ----------
    y : np.ndarray
        Time series data
    start : int
        Start index (inclusive)
    end : int
        End index (exclusive)
    
    Returns
    -------
    float
        Sum of squared deviations from segment mean
    """
    if end <= start:
        return 0.0
    segment = y[start:end]
    return np.sum((segment - segment.mean()) ** 2)


def cusum_detector(
    y: np.ndarray,
    threshold: float = 5.0,
    drift: float = 0.5,
    target_mean: float | None = None
) -> tuple[np.ndarray, list[int]]:
    """
    CUSUM (Cumulative Sum) change point detector.
    
    Detects upward shifts in mean using: S_t = max(0, S_{t-1} + (x_t - μ₀) - k)
    
    Parameters
    ----------
    y : np.ndarray
        Time series data
    threshold : float
        Detection threshold h; alarm when S_t > h
    drift : float
        Allowable drift k (slack parameter)
    target_mean : float, optional
        Target mean μ₀; uses first 20% of data if None
    
    Returns
    -------
    cusum_stats : np.ndarray
        CUSUM statistic at each time point
    change_points : list[int]
        Detected change point indices
    """
    n = len(y)
    if target_mean is None:
        # Estimate from initial portion
        target_mean = np.mean(y[: max(1, n // 5)])
    
    cusum_stats = np.zeros(n)
    change_points = []
    
    for t in range(1, n):
        cusum_stats[t] = max(0.0, cusum_stats[t - 1] + (y[t] - target_mean) - drift)
        if cusum_stats[t] > threshold and (not change_points or t - change_points[-1] > 10):
            change_points.append(t)
            # Reset after detection
            cusum_stats[t] = 0.0
    
    return cusum_stats, change_points


def binary_segmentation(
    y: np.ndarray,
    min_size: int = 10,
    penalty: float = 10.0,
    start: int = 0,
    end: int | None = None
) -> list[int]:
    """
    Binary segmentation for change point detection.
    
    Recursively splits the series at the point minimizing total cost,
    stopping when the cost reduction is below the penalty.
    
    Parameters
    ----------
    y : np.ndarray
        Time series data
    min_size : int
        Minimum segment size
    penalty : float
        Penalty per change point (BIC-style regularization)
    start : int
        Start index of current segment
    end : int, optional
        End index of current segment
    
    Returns
    -------
    list[int]
        Sorted list of detected change point indices
    """
    if end is None:
        end = len(y)
    
    if end - start < 2 * min_size:
        return []
    
    # Current cost without split
    base_cost = segment_cost(y, start, end)
    
    # Find best split point
    best_split = None
    best_cost = base_cost
    
    for tau in range(start + min_size, end - min_size + 1):
        split_cost = segment_cost(y, start, tau) + segment_cost(y, tau, end) + penalty
        if split_cost < best_cost:
            best_cost = split_cost
            best_split = tau
    
    if best_split is None:
        return []
    
    # Recurse on both sides
    left_cps = binary_segmentation(y, min_size, penalty, start, best_split)
    right_cps = binary_segmentation(y, min_size, penalty, best_split, end)
    
    return sorted(left_cps + [best_split] + right_cps)


def zscore_anomaly_detector(
    y: np.ndarray,
    threshold: float = 3.0,
    window: int = 20
) -> tuple[np.ndarray, np.ndarray]:
    """
    Z-score based point anomaly detector with rolling statistics.
    
    Computes z_t = (y_t - μ_window) / σ_window and flags |z_t| > threshold.
    
    Parameters
    ----------
    y : np.ndarray
        Time series data
    threshold : float
        Z-score threshold for anomaly detection (typically 2-3)
    window : int
        Rolling window size for mean and std computation
    
    Returns
    -------
    z_scores : np.ndarray
        Z-score at each point (NaN for initial window)
    anomaly_indices : np.ndarray
        Indices of detected anomalies
    """
    n = len(y)
    z_scores = np.full(n, np.nan)
    
    for t in range(window, n):
        window_data = y[t - window : t]
        mu = window_data.mean()
        sigma = window_data.std()
        if sigma > 1e-10:
            z_scores[t] = (y[t] - mu) / sigma
    
    anomaly_mask = np.abs(z_scores) > threshold
    anomaly_indices = np.where(anomaly_mask)[0]
    
    return z_scores, anomaly_indices


print("✓ NumPy implementations loaded: segment_cost, cusum_detector, binary_segmentation, zscore_anomaly_detector")

## Plotly Visualizations

Interactive visualizations demonstrating each algorithm on synthetic data.

In [ ]:
# Generate synthetic data with regime shifts
rng = np.random.default_rng(42)
n_points = 300
t = np.arange(n_points)

# Base signal with mean shifts
y_demo = rng.normal(0, 0.5, n_points)
y_demo[100:200] += 2.5  # Regime 2
y_demo[200:] += 1.0     # Regime 3
true_cps = [100, 200]

# Add point anomalies
anomaly_idx = np.array([50, 150, 250])
y_demo[anomaly_idx] += np.array([4.0, -3.5, 5.0])

print(f"Generated series: {n_points} points, true change points at {true_cps}")

In [ ]:
# CUSUM Change Point Detection Visualization
cusum_stats, cusum_cps = cusum_detector(y_demo, threshold=8.0, drift=0.3)

from plotly.subplots import make_subplots

fig_cusum = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Time Series with CUSUM Detections", "CUSUM Statistic Evolution"),
    vertical_spacing=0.12,
    row_heights=[0.5, 0.5]
)

# Top: Original series with detected change points
fig_cusum.add_trace(
    go.Scatter(x=t, y=y_demo, mode="lines", name="Series", line=dict(color="steelblue")),
    row=1, col=1
)
for cp in cusum_cps:
    fig_cusum.add_vline(x=cp, line_dash="dash", line_color="red", row=1, col=1)
    fig_cusum.add_annotation(
        x=cp, y=y_demo.max(), text=f"CP={cp}", showarrow=False,
        font=dict(size=10, color="red"), row=1, col=1
    )

# Mark true change points
for tcp in true_cps:
    fig_cusum.add_vline(x=tcp, line_dash="dot", line_color="green", opacity=0.5, row=1, col=1)

# Bottom: CUSUM statistic with threshold
fig_cusum.add_trace(
    go.Scatter(x=t, y=cusum_stats, mode="lines", name="CUSUM S_t", line=dict(color="purple")),
    row=2, col=1
)
fig_cusum.add_hline(y=8.0, line_dash="dash", line_color="orange", row=2, col=1,
                    annotation_text="threshold h=8")

fig_cusum.update_layout(
    title="CUSUM Change Point Detection: S_t = max(0, S_{t-1} + (x_t - μ) - k)",
    height=500,
    showlegend=True,
    margin=dict(l=40, r=20, t=60, b=40)
)
fig_cusum.update_xaxes(title_text="Time", row=2, col=1)
fig_cusum.update_yaxes(title_text="Value", row=1, col=1)
fig_cusum.update_yaxes(title_text="CUSUM", row=2, col=1)
fig_cusum.show()

In [ ]:
# Binary Segmentation Visualization
binseg_cps = binary_segmentation(y_demo, min_size=15, penalty=20.0)

# Create color-coded segments
segment_colors = ["#3498db", "#e74c3c", "#2ecc71", "#9b59b6", "#f39c12"]
all_cps = [0] + binseg_cps + [len(y_demo)]

fig_seg = go.Figure()

for i in range(len(all_cps) - 1):
    start, end = all_cps[i], all_cps[i + 1]
    color = segment_colors[i % len(segment_colors)]
    fig_seg.add_trace(
        go.Scatter(
            x=t[start:end],
            y=y_demo[start:end],
            mode="lines",
            name=f"Segment {i+1}",
            line=dict(color=color, width=2),
            fill="tozeroy",
            fillcolor=color.replace(")", ", 0.2)").replace("rgb", "rgba") if "rgb" in color else f"rgba{tuple(int(color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) + (0.15,)}"
        )
    )

# Mark detected change points
for cp in binseg_cps:
    fig_seg.add_vline(x=cp, line_dash="solid", line_color="black", line_width=2)
    fig_seg.add_annotation(
        x=cp, y=y_demo.max() + 0.5,
        text=f"τ={cp}",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowcolor="black",
        font=dict(size=11, color="black")
    )

# Mark true change points for comparison
for tcp in true_cps:
    fig_seg.add_vline(x=tcp, line_dash="dot", line_color="gray", opacity=0.6)

fig_seg.update_layout(
    title=f"Binary Segmentation: Detected {len(binseg_cps)} change points (true: {true_cps})",
    xaxis_title="Time",
    yaxis_title="Value",
    height=400,
    margin=dict(l=40, r=20, t=60, b=40),
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)
fig_seg.show()

print(f"Detected change points: {binseg_cps}")
print(f"Segment costs: {[round(segment_cost(y_demo, all_cps[i], all_cps[i+1]), 2) for i in range(len(all_cps)-1)]}")

In [ ]:
# Z-Score Anomaly Detection Visualization
z_scores, detected_anomalies = zscore_anomaly_detector(y_demo, threshold=2.5, window=25)

fig_anom = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Time Series with Detected Anomalies", "Z-Score with Detection Threshold"),
    vertical_spacing=0.12,
    row_heights=[0.5, 0.5]
)

# Top: Series with anomaly markers
fig_anom.add_trace(
    go.Scatter(x=t, y=y_demo, mode="lines", name="Series", line=dict(color="steelblue")),
    row=1, col=1
)
fig_anom.add_trace(
    go.Scatter(
        x=detected_anomalies,
        y=y_demo[detected_anomalies],
        mode="markers",
        name="Detected Anomalies",
        marker=dict(color="red", size=12, symbol="x")
    ),
    row=1, col=1
)
# Mark true anomalies
fig_anom.add_trace(
    go.Scatter(
        x=anomaly_idx,
        y=y_demo[anomaly_idx],
        mode="markers",
        name="True Anomalies",
        marker=dict(color="green", size=10, symbol="circle-open", line=dict(width=2))
    ),
    row=1, col=1
)

# Bottom: Z-scores with threshold bands
fig_anom.add_trace(
    go.Scatter(x=t, y=z_scores, mode="lines", name="Z-score", line=dict(color="purple")),
    row=2, col=1
)
fig_anom.add_hline(y=2.5, line_dash="dash", line_color="red", row=2, col=1)
fig_anom.add_hline(y=-2.5, line_dash="dash", line_color="red", row=2, col=1)
fig_anom.add_hrect(y0=-2.5, y1=2.5, fillcolor="green", opacity=0.1, row=2, col=1,
                   annotation_text="Normal zone", annotation_position="top left")

# Mark anomalies on z-score plot
fig_anom.add_trace(
    go.Scatter(
        x=detected_anomalies,
        y=z_scores[detected_anomalies],
        mode="markers",
        name="Anomaly Z-scores",
        marker=dict(color="red", size=8),
        showlegend=False
    ),
    row=2, col=1
)

fig_anom.update_layout(
    title="Z-Score Anomaly Detection: z_t = (y_t - μ) / σ, anomaly if |z_t| > τ",
    height=520,
    margin=dict(l=40, r=20, t=60, b=40)
)
fig_anom.update_xaxes(title_text="Time", row=2, col=1)
fig_anom.update_yaxes(title_text="Value", row=1, col=1)
fig_anom.update_yaxes(title_text="Z-score", row=2, col=1)
fig_anom.show()

print(f"Detected anomalies at indices: {detected_anomalies.tolist()}")
print(f"True anomaly indices: {anomaly_idx.tolist()}")
print(f"Precision: {len(set(detected_anomalies) & set(anomaly_idx)) / max(1, len(detected_anomalies)):.2f}")

### Algorithm Comparison Summary

| Algorithm | Use Case | Complexity | Key Parameter |
|-----------|----------|------------|---------------|
| **CUSUM** | Online mean shift detection | $O(n)$ | Threshold $h$, drift $k$ |
| **Binary Segmentation** | Offline multiple change points | $O(n \log n)$ | Penalty $\beta$, min segment size |
| **Z-Score** | Point anomaly detection | $O(n)$ | Threshold $\tau$, window size |

**Trade-offs:**
- CUSUM is sequential and resets after detection—good for streaming
- Binary segmentation finds globally optimal splits but requires full data
- Z-score is simple but assumes local stationarity within windows

## Next steps
- Explore the dynamic catalog in `data_science/time_series/sktime_algorithms/registry/06_annotation_catalog.ipynb`.
- Combine annotators with transformers (detrending, smoothing) for cleaner signals.